In [2]:
import vk
import re
import requests
import time
from datetime import datetime, timedelta

In [3]:
vkapi = vk.API(vk.Session(), v='5.20', lang='ru', timeout=100)

In [4]:
def get_photo_ids(geohash='8NTZtoXr'):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:28.0) Gecko/20100101 Firefox/28.0',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest'
    }

    data = {
        "act": "show_photo_more",
        "al": 1,
        "geohash": geohash,
        "offset": 0,
        "photo_skip": '0_',
    }

    content = requests.Session().post('https://vk.com/al_places.php', data=data, headers=headers)
    html = content.content.decode('utf-8')
    prog = re.compile('href="\/photo(\w+)"')
    photo_ids = set(prog.findall(html))
                    
    return photo_ids

In [ ]:
photo_ids = get_photo_ids()
print(list(photo_ids)[:10])
print(len(photo_ids))

Вывод: возвращается всегда 120 фоток

In [5]:
def get_photos(photo_ids):
    return vkapi.photos.getById(photos=photo_ids)

In [14]:
photos = get_photos(photo_ids = ','.join(photo_ids))

print(photos[0])

{'long': 30.325557, 'album_id': 233537241, 'id': 422786129, 'text': '', 'photo_75': 'http://cs630728.vk.me/v630728014/387d1/r-NG6q48B78.jpg', 'photo_1280': 'http://cs630728.vk.me/v630728014/387d5/IW_WOOQ1a-k.jpg', 'owner_id': 89407014, 'photo_130': 'http://cs630728.vk.me/v630728014/387d2/HbrQk31R8MU.jpg', 'date': 1467716269, 'photo_807': 'http://cs630728.vk.me/v630728014/387d4/d3RiNoM0K9w.jpg', 'lat': 59.934385, 'width': 960, 'photo_604': 'http://cs630728.vk.me/v630728014/387d3/PYfrtPH-USY.jpg', 'height': 719}


In [6]:
photos_with_post = list(filter(lambda p: 'post_id' in p, photos))

In [7]:
print(len(photos), len(photos_with_post))

120 8


Вывод: фотки с постами лишь малая часть от всех

In [6]:
first_photo_ids = get_photo_ids()
while True:
    photo_ids = get_photo_ids()
    print("{} количество первых фоток = {}, текущих = {}, пересечение фоток = {}.".
             format(datetime.now(),len(first_photo_ids),len(photo_ids), len(first_photo_ids.intersection(photo_ids))))
    
    time.sleep(30)

2016-07-07 15:23:11.982524 количество первых фоток = 120, текущих = 120, пересечение фоток = 120.
2016-07-07 15:23:43.070168 количество первых фоток = 120, текущих = 120, пересечение фоток = 120.
2016-07-07 15:24:14.402744 количество первых фоток = 120, текущих = 120, пересечение фоток = 120.
2016-07-07 15:24:45.686226 количество первых фоток = 120, текущих = 120, пересечение фоток = 120.
2016-07-07 15:25:16.902538 количество первых фоток = 120, текущих = 120, пересечение фоток = 118.
2016-07-07 15:25:48.984717 количество первых фоток = 120, текущих = 120, пересечение фоток = 118.
2016-07-07 15:26:20.322562 количество первых фоток = 120, текущих = 120, пересечение фоток = 118.
2016-07-07 15:26:51.416558 количество первых фоток = 120, текущих = 120, пересечение фоток = 118.
2016-07-07 15:27:22.605655 количество первых фоток = 120, текущих = 120, пересечение фоток = 118.
2016-07-07 15:27:54.227047 количество первых фоток = 120, текущих = 120, пересечение фоток = 118.
2016-07-07 15:28:25.

KeyboardInterrupt: 

Вывод: 120 фоток добавилось за 20 минут

In [6]:
geohashes = ['8NTZtoXr',
'8NTZtoc8',
'8NTZto0w',
'8NTZtoBO',
'8NTZtoHQ',
'8NTZtoTK',
'8NTZtoWk',
'8NTZtobF',
'8NTZtowS',
'8NTZtoy6',
'8NTZtom9',
'8NTZtoOf',
'8NTZtoNc',
'8NTZtoGl',
'8NTZtoLx',
'8NTZtoid',
'8NTZtoKc',
'8NTZtoIc',
'8NTZtoCA',
'8NTZs9yp',
'8NTZs9t0',
'8NTZs9va',
'8NTZs_G1',
'8NTZs9wm',
'8NTZs9ae',
'8NTZs9TS',
'8NTZs3-g',
'8NTZs9nJ',
'8NTZs9ga',
'8NTZs8N_',
'8NTZs8nE',
'8NTZs8uT',
'8NTZs-hl',
'8NTZs-mV',
'8NTZs-2r',
'8NTZs-U_',
'8NTZs3_6',
'8NTZs9c9',
'8NTZs9ev']

In [35]:
all_photo_ids = set()
all_photos_with_post = set()
photos_count = 0

for geohash in geohashes:
    try:
        photo_ids = get_photo_ids(geohash)
        photos_count += len(photo_ids)
        all_photo_ids.update(photo_ids)
        photos = get_photos(photo_ids = ','.join(photo_ids))
        posts = ['{}_{}'.format(p['owner_id'], p['post_id']) for p in photos if 'post_id' in p.keys()]

        all_photos_with_post.update(posts)

        print("геохеш = {} максимум {}, {} добавлено, {} всего фото, всего с постами {}".format(
                                                                                   geohash,
                                                                                   photos_count,
                                                                                   len(photo_ids),
                                                                                   len(all_photo_ids),
                                                                                   len(all_photos_with_post)))  
    except Exception as e:
        print(e)
        
    


геохеш = 8NTZtoXr максимум 120, 120 добавлено, 120 всего фото, всего с постами 8
геохеш = 8NTZtoc8 максимум 240, 120 добавлено, 129 всего фото, всего с постами 10
геохеш = 8NTZto0w максимум 360, 120 добавлено, 142 всего фото, всего с постами 11
геохеш = 8NTZtoBO максимум 480, 120 добавлено, 220 всего фото, всего с постами 15
геохеш = 8NTZtoHQ максимум 600, 120 добавлено, 225 всего фото, всего с постами 15
геохеш = 8NTZtoTK максимум 720, 120 добавлено, 225 всего фото, всего с постами 15
геохеш = 8NTZtoWk максимум 840, 120 добавлено, 225 всего фото, всего с постами 15
геохеш = 8NTZtobF максимум 960, 120 добавлено, 227 всего фото, всего с постами 16
геохеш = 8NTZtowS максимум 1080, 120 добавлено, 233 всего фото, всего с постами 16
геохеш = 8NTZtoy6 максимум 1200, 120 добавлено, 237 всего фото, всего с постами 16
геохеш = 8NTZtom9 максимум 1320, 120 добавлено, 243 всего фото, всего с постами 18
геохеш = 8NTZtoOf максимум 1440, 120 добавлено, 245 всего фото, всего с постами 18
геохеш = 8NTZ

Вывод: постов с фото очень мало

In [9]:
tags300 = ['8NTZEo1h',
'8NTZEpiU',
'8NTZEpmw',
'8NTZEpn_',
'8NTZEp8m',
'8NTZEsqZ',
'8NTZEsu_',
'8NTZEuRh',
'8NTZEuXa',
'8NTZEtio',
'8NTZEnqx',
'8NTZEniW',
'8NTZElvn',
'8NTZEnXG',
'8NTZElNu',
'8NTZEP7L',
'8NTZEPpi',
'8NTZEOzr',
'8NTZEOmt',
'8NTZEL2N',
'8NTZEL7W',
'8NTZEkAU',
'8NTZEkQu',
'8NTZEkXn',
'8NTZEk0v',
'8NTZEknE',
'8NTZEhfL',
'8NTZEhn8',
'8NTZEh8W',
'8NTZEksz',
'8NTZEk8d',
'8NTZEmbF',
'8NTZEmD-',
'8NTZEjVI',
'8NTZEhvs',
'8NTZEjG8',
'8NTZEjZD',
'8NTZEj1z',
'8NTZEmmo',
'8NTZEm-B',
'8NTZEsbn',
'8NTZEsnO',
'8NTZEshp',
'8NTZEpV9',
'8NTZEj4x',
'8NTZEpN4']


In [60]:
alphabet = set(map(chr, range( ord('a'), ord('z')+1 )))
alphabet.update(set(map(chr, range( ord('A'), ord('Z')+1 ))))
alphabet.update(set(map(chr, range( ord('0'), ord('9')+1 ))))
alphabet.update({'-','_'})
alphabet

all_tags = set()
for c1 in alphabet:
    for c2 in alphabet:
        for c3 in alphabet:
            all_tags.add("8NTZE"+c1+c2+c3)
            

print(len(all_tags))

262144


In [62]:
all_photo_ids = set()
all_photos_with_post = set()
photos_count = 0

for geohash in all_tags:
    try:
        photo_ids = get_photo_ids(geohash)
        photos_count += len(photo_ids)
        all_photo_ids.update(photo_ids)
        photos = get_photos(photo_ids = ','.join(photo_ids))
        posts = ['{}_{}'.format(p['owner_id'], p['post_id']) for p in photos if 'post_id' in p.keys()]

        all_photos_with_post.update(posts)

        print("{} геохеш = {} максимум {}, {} добавлено, {} всего фото, всего с постами {}".format(
                                                                                   datetime.now(),
                                                                                   geohash,
                                                                                   photos_count,
                                                                                   len(photo_ids),
                                                                                   len(all_photo_ids),
                                                                                   len(all_photos_with_post)))  
    except Exception as e:
        print(e)
        

2016-07-08 12:01:41.038418 геохеш = 8NTZEykG максимум 120, 120 добавлено, 120 всего фото, всего с постами 11
2016-07-08 12:01:42.133189 геохеш = 8NTZEDOj максимум 240, 120 добавлено, 240 всего фото, всего с постами 21
2016-07-08 12:01:43.318505 геохеш = 8NTZEpNi максимум 360, 120 добавлено, 360 всего фото, всего с постами 35
2016-07-08 12:01:44.408489 геохеш = 8NTZEhJ9 максимум 480, 120 добавлено, 480 всего фото, всего с постами 47
2016-07-08 12:01:44.590121 геохеш = 8NTZE9Ax максимум 513, 33 добавлено, 513 всего фото, всего с постами 50
2016-07-08 12:01:45.627032 геохеш = 8NTZEIBW максимум 633, 120 добавлено, 632 всего фото, всего с постами 70
2016-07-08 12:01:46.601378 геохеш = 8NTZEwWT максимум 753, 120 добавлено, 698 всего фото, всего с постами 82
2016-07-08 12:01:47.551923 геохеш = 8NTZEv6y максимум 873, 120 добавлено, 818 всего фото, всего с постами 93
2016-07-08 12:01:47.787788 геохеш = 8NTZE2eG максимум 928, 55 добавлено, 859 всего фото, всего с постами 95
2016-07-08 12:01:48.0

KeyboardInterrupt: 

Вывод: можно получить больше фоток, стоит параллелить

In [44]:
s1 = get_photo_ids('8NTZEpii')
s2 = get_photo_ids('8NTZEpik')
print(len(s1), len(s2), len(s1.intersection(s2)))

120 120 118


In [21]:
def get_photo_url(p):
    sizes = list(filter(lambda p: p.startswith('photo'), p.keys()))
    maxsize = max(list(map(lambda p: int(p.replace("photo_",'')), sizes)))
  
    return p['photo_' + str(maxsize)]

In [26]:
for tag in tags300:
    print(tag)
    photo_ids = get_photo_ids(tag)
    photos = get_photos(photo_ids = ','.join(photo_ids))
 
    with open('posts.csv', 'a') as f:
        for p in photos:
            f.write('{},{},{},{}\n'.format(tag, p['lat'], p['long'], get_photo_url(p)))

8NTZEo1h
фоток с постами 16
8NTZEpiU
фоток с постами 23
8NTZEpmw
фоток с постами 19
8NTZEpn_
фоток с постами 18
8NTZEp8m
фоток с постами 11
8NTZEsqZ
фоток с постами 15
8NTZEsu_


KeyboardInterrupt: 

можно ли с квадрата получить больше 800 фоток за 20 минут?

In [63]:
import pymongo


vk_posts_collection = pymongo.MongoClient(host="192.168.13.110").VkFest.VkFest2016_posts

for p in vk_posts_collection.find().limit(100):
    print(p)

сколько набежит постов за выходные